In [ ]:
!pip install -q crewai crewai_tools langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.3/701.3 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.6 MB/s eta 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["SERPER_API_KEY"] = ""

In [ ]:
# User input
# - Location
# - Duration
# - Budget
# - Startdate
# - num of people

# Travel Plan
# Sightseeing - top attractions
# fooding - food options / suggestion
# hotel - where  which hotels
# family friendly / kids friendly place
# ITINERY - Day by Day

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
search_tool = SerperDevTool() #The SerperDevTool is a software component that performs internet searches using the Serper.dev API, a third-party service that delivers structured Google search results.
scrape_tool = ScrapeWebsiteTool() #

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
# From tools import results of search trip and scrap page

def get_travel_researcher():
  return Agent(
      role = "Travel Researcher",
      goal = "Find best attractions, accomodations, travel tips",
      backstory = "Expert in finding travel destinations , must visit spots",
      tools = [scrape_tool,search_tool],
      verbose = True
      #llm = ?
  )

def get_travel_planner():
  return Agent(
      role = "Travel Planner",
      goal = "Create a day by day travel itinery",
      backstory = "Expert in creating enjoyable , family friendy amazing travel plans",
      verbose = True
  )

def get_budget_Analyst():
   return Agent(
      role = "Budget Controller",
      goal = "Check if itinery fits the budget and suggest imporvements. Get the costs for the hotels on the specific dates mentioned and latest costs from latest websites",
      backstory = "Expert in cost estimation and travel budget optimization ensuring travel plans stay within budget",
      verbose = True
  )

# User input
user_input ={
    "location": "Singapore",
    "duration": 5,
    "budget" : 200000,
    "start_date":"2025-10-01",
    "num_people":4
}


# Agents
researcher = get_travel_researcher()
planner = get_travel_planner()
budget_analyst = get_budget_Analyst()

# Tasks
research_task = Task(
    description = f""" Use search and scraping tools to find
    - top attractions
    - family/ kid friendly places
    - best hotels / best hotel areas
    - travel tips
    - local food suggestions for {user_input['location']}
    """,
    expected_output = "A bullet point summary of attractions, hotels or hotel areas and travel tips along with food suggestions or food places",
    agent = researcher
)

planning_task = Task(
    description = f"""Create a {user_input['duration']} day itinery for {user_input['location']} starting from {user_input['start_date']} for {user_input['num_people']} people
    using the research findings. Include specifically sightseeing, rest time , hotel stay , food option details.
    """,
    expected_output = "A day-wise itinery with optional notes for each sighseeing or activity proposed",
    agent = planner
)

budget_optimization_task = Task(
    description = f"""Estimate the total travel cost in INR based on the itinery propsed. Allowed budget is INR{user_input['budget']} for {user_input['num_people']} people.
    Suggest cost cutting alternatives to ensure the itinery stays within budget if the proposed itinery exceeds budget
    """,
    expected_output = "Total cost estimate and suggestions to reduce cost to fit budget of needed",
    agent = budget_analyst
    # we can introduce tools for accurate calculation . Right now LLM is being used for calculation - result may or may not be exact
)


#Crew
crew = Crew(
    agents = [researcher,planner,budget_analyst],
    tasks = [research_task,planning_task,budget_optimization_task],
    verbose = True
)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 248b4a6b-e4b3-4ca8-88aa-1a13dba75f45                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Researcher                                                                                       │
│                                                                                                                 │
│  Task:  Use search and scraping tools to find                                                                   │
│      - top attractions                                                                                          │
│      - family/ kid friendly places                                                                              │
│      - best hotels / best hotel areas                                                                           │
│      - travel tips                                                                                              │
│      - local food suggestions for Singapore                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: 
datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects 
to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

...

Output()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Budget Controller                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Total Travel Cost Estimate for Singapore Trip (October 1 - October 5, 2025) for 4 People                   │
│                                                                                                                 │
│  #### Hotel Costs:                                                                                              │
│  1. **Marina Bay Sands**:                                                                                       │
│     - Average rate: INR 30,000 per night (for a family room)                                                    │
│     - Total for 4 nights: INR 30,000 x 4 = **INR 120,000**                                                      │
│                                                                                                                 │
│  #### Attraction Costs:                                                                                         │
│  1. **Gardens by the Bay**:                                                                                     │
│     - Entry Fee: INR 2,000 per person                                                                           │
│     - Total: 4 x INR 2,000 = **INR 8,000**                                                                      │
│                                                                                                                 │
│  2. **Universal Studios Singapore**:                                                                            │
│     - Ticket Price: INR 5,500 per person                                                                        │
│     - Total: 4 x INR 5,500 = **INR 22,000**                                                                     │
│                                                                                                                 │
│  3. **Singapore Zoo**:                                                                                          │
│     - Ticket Price: INR 2,500 per person                                                                        │
│     - Total: 4 x INR 2,500 = **INR 10,000**                                                                     │
│                                                                                                                 │
│  4. **Night Safari**:                                                                                           │
│     - Ticket Price: INR 3,000 per person                                                                        │
│     - Total: 4 x INR 3,000 = **INR 12,000**                                                                     │
│                                                                                                                 │
│  5. **Adventure Cove Waterpark**:                                                                               │
│     - Ticket Price: INR 3,000 per person                                                                        │
│     - Total: 4 x INR 3,000 = **INR 12,000**                                                                     │
│                                                                                                                 │
│  6. **S.E.A. Aquarium**:                                                                                        │
│     - Ticket Price: INR 3,500 per person               

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d35d0e56-1f3c-42e4-8e90-cbc7a5ea9e41                                                                     │
│  Agent: Budget Controller                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 248b4a6b-e4b3-4ca8-88aa-1a13dba75f45                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Total Travel Cost Estimate for Singapore Trip (October 1 - October 5, 2025) for 4 People     │
│                                                                                                                 │
│  #### Hotel Costs:                                                                                              │
│  1. **Marina Bay Sands**:                                                                                       │
│     - Average rate: INR 30,000 per night (for a family room)                                                    │
│     - Total for 4 nights: INR 30,000 x 4 = **INR 120,000**                                                      │
│                                                                                                                 │
│  #### Attraction Costs:                                                                                         │
│  1. **Gardens by the Bay**:                                                                                     │
│     - Entry Fee: INR 2,000 per person                                                                           │
│     - Total: 4 x INR 2,000 = **INR 8,000**                                                                      │
│                                                                                                                 │
│  2. **Universal Studios Singapore**:                                                                            │
│     - Ticket Price: INR 5,500 per person                                                                        │
│     - Total: 4 x INR 5,500 = **INR 22,000**                                                                     │
│                                                                                                                 │
│  3. **Singapore Zoo**:                                                                                          │
│     - Ticket Price: INR 2,500 per person                                                                        │
│     - Total: 4 x INR 2,500 = **INR 10,000**                                                                     │
│                                                                                                                 │
│  4. **Night Safari**:                                                                                           │
│     - Ticket Price: INR 3,000 per person                                                                        │
│     - Total: 4 x INR 3,000 = **INR 12,000**                                                                     │
│                                                                                                                 │
│  5. **Adventure Cove Waterpark**:                                                                               │
│     - Ticket Price: INR 3,000 per person                                                                        │
│     - Total: 4 x INR 3,000 = **INR 12,000**                                                                     │
│                                                                                                                 │
│  6. **S.E.A. Aquarium**:                              

In [ ]:
from IPython.display import display, Markdown, HTML
Markdown(result.raw)

### Total Travel Cost Estimate for 4 People in INR

1. **Accommodation Costs:**
   - **Marina Bay Sands** (5 days): Approx. INR 40,000 per night
   - Total for 5 nights: 5 * 40,000 = **INR 200,000**

2. **Food Costs:**
   - Breakfast at hotel: (included in hotel rate)
   - Lunch (5 days average INR 1,500 per meal): 5 * 1,500 = **INR 7,500**
   - Dinner (5 days average INR 2,500 per meal): 5 * 2,500 = **INR 12,500**
   - Total for meals: **INR 20,000**

3. **Attraction Costs:**
   - **Gardens by the Bay:** INR 1,000 (entry for 4)
   - **Universal Studios Singapore:** Approx. INR 12,000 (INR 3,000 each for 4)
   - **Singapore Zoo:** Approx. INR 4,000 (INR 1,000 each for 4)
   - **S.E.A. Aquarium:** Approx. INR 4,000 (INR 1,000 each for 4)
   - **Adventure Cove Waterpark:** Approx. INR 4,000 (INR 1,000 each for 4)
   - Total for attractions: **INR 25,000**

4. **Transport Costs:**
   - MRT passes (average INR 200 per day): 5 days * 4 people * 200 = **INR 4,000**

Adding all these together:
- **Accommodation:** INR 200,000
- **Food:** INR 20,000
- **Attractions:** INR 25,000
- **Transport:** INR 4,000

**Total Estimated Cost: INR 249,000**

### Budget and Improvements:
The proposed itinerary exceeds the budget of **INR 200,000** by **INR 49,000**. Here are the cost-cutting alternatives to fit within the budget:

1. **Accommodation:** 
   - Instead of Marina Bay Sands, consider staying in **Chinatown** or **Clarke Quay** where hotels can average around INR 15,000 to INR 20,000 per night, potentially reducing the total hotel cost to around **INR 75,000** for 5 nights.

2. **Food:**
   - Opt for hawker centers for all meals instead of some restaurant dinners. This could lower the meal budget greatly. Averages could go down to **INR 10,000** total for the 5 days if meals average closer to **INR 500 per meal**.

3. **Attractions:**
   - Consider choosing 3 key attractions instead of all listed or use a combo ticket for some of the more child-friendly places which may reduce entry fees, potentially lowering this category to around **INR 15,000**.

4. **Transport:**
   - Reduce transport costs by leveraging more walks, especially in areas like Marina Bay, which are pedestrian-friendly. This could cut down transport costs by half, leaving roughly **INR 2,000**.

---

### Implementing Improvements

- **New Accommodation Costs:** INR 75,000
- **New Food Costs:** INR 10,000
- **New Attraction Costs:** INR 15,000
- **New Transport Costs:** INR 2,000

**Adjusted Total Cost: INR 102,000**

By selecting budget-friendly hotels, focusing on local food, and carefully choosing attractions, the total could reduce to around **INR 102,000**, keeping it comfortably under your INR 200,000 budget while still ensuring a rich travel experience. 

Happy planning for your trip to Singapore! Enjoy your adventure!

In [ ]:
# Ambiguous Ask - Iterations within ourselves ...( Autogen)
# I need to do an RCA of why sales went down?
# HYpothesis - a) Product category b) Region c) Product x Region d) App issue